In [ ]:
import pandas as pd
# Load our dataset 
df_yelp = pd.read_table('yelp_labelled.txt') 
df_imdb = pd.read_table('imdb_labelled.txt') 
df_amz = pd.read_table('amazon_cells_labelled.txt')



FileNotFoundError: ignored

In [ ]:
# Concatenate our Datasets 
frames = [df_yelp, df_imdb, df_amz] 
frames

In [ ]:
# Renaming Column Headers
for colname in frames:
  colname.columns = ["Message", "Target"]
frames

NameError: ignored

In [ ]:
#Assign a Key
keys = ['Yelp', 'IMDB', 'Amazon']

In [ ]:
#Merge or Concat our Datasets
df = pd.concat(frames, keys = keys)
df

#Data Cleaning

In [7]:
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create List of punctuation marks
punctuations = string.punctuation

# Create out list of Stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

#load English Tokenizer
parser = English()

# Creating Tokenizer Func
def spacy_tokenizer(sentence):
  #Create token object
  mytokens = parser(sentence)

  #Lemmatizing each token and covert to lowercase
  mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

  #Removing Stop words
  mytokens = [word for word in mytokens if word not in stop_words and word not in punctuations ]

  return mytokens

In [ ]:
Fcoimport string 
import spacy 
from spacy.lang.en.stop_words import STOP_WORDS 
from spacy.lang.en import English

# Create our list of punctuation marks 
punctuations = string.punctuation

# Create our list of stopwords 
nlp = spacy.load('en_core_web_sm') 
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors 
parser = English()

# Creating our tokenizer function 
def spacy_tokenizer (sentence):
  # Creating our token object, which is used to create documents with linguistic annotations. 
  mytokens = parser (sentence)

  # Lemmatizing each token and converting each token into lowercase 
  mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

  #Removing stop words 
  mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

  # return preprocessed list of tokens 
  return mytokens

#Defining a Transformer


In [9]:
from sklearn.base import TransformerMixin

# This function will clean the text 
def clean_text (text):
  return text.strip().lower()
#Custom transformer using Python standard library (you could use spacy as well) 
class predictors (TransformerMixin):

  def transform(self, x, **transform_params):
    return [clean_text (text) for text in X]

  def fit(self, x, y=None, **fit_params) :
    return self
    
  def get_params (self, deep=True):
    return { }

#Feature Engineering

###Bag of Words

In [10]:
from sklearn. feature_extraction.text import CountVectorizer
#vectorizer
#We create our bag of words (bow) using our tokenizer and defining an ngram range 
bow = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))

###TF-IDF

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

#tfidf
tfvectorizer = TfidfVectorizer(tokenizer = spacy_tokenizer)

#Train and Test Split

In [12]:
X = df['Message']
ylabels = df['Target']

NameError: ignored

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2, random_state=0)

NameError: ignored

#Classifier

In [14]:
from sklearn.neural_network import MLPClassifier
#classifier
classifier_MLP = MLPClassifier(max_iter=400, hidden_layer_sizes=(100,2), verbose = True)

#Create Pipeline

In [15]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([("cleaner", predictors()), ('vectorizer', bow), ('classifier', classifier_MLP)], verbose = True)

In [ ]:
#fit data
pipe.fit(X_train, y_train)

In [ ]:
#Evaluate

In [ ]:
sample_prediction = pipe.predict(X_test)

In [ ]:
from sklearn import model_selection
accuracy = metrics.accuracy_score(y_test, sample_prediction)
precision = metrics.precision_score(y_test, sample_prediction)
recall = metrics.recall_score(y_test, sample_prediction)

In [ ]:
!pip install -q gradio
!pip install -q transformers

     |████████████████████████████████| 5.1 MB 21.5 MB/s 
     |████████████████████████████████| 84 kB 600 kB/s 
     |████████████████████████████████| 212 kB 54.3 MB/s 
     |████████████████████████████████| 256 kB 55.4 MB/s 
     |████████████████████████████████| 2.0 MB 48.5 MB/s 
     |████████████████████████████████| 54 kB 2.3 MB/s 
     |████████████████████████████████| 53 kB 2.3 MB/s 
     |████████████████████████████████| 1.1 MB 48.0 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
     |████████████████████████████████| 144 kB 42.7 MB/s 
     |████████████████████████████████| 271 kB 28.8 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 11.1 MB 50.6 MB/s 
     |████████████████████████████████| 80 kB 9.1 MB/s 
     |████████████████████████████████| 43 kB 984 kB/s 
     |████████████████████████████████| 4.0 MB 6.0 MB/s 
     |████████████████████████████████| 856 kB 47.3 MB/s 
     |██████████████████████

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import gradio as gr
from transformers import pipeline

sentiment=pipeline("sentiment-analysis")

def get_sentiment(text):
  txt_split = text.split(";")
  data = {'text': [], 'label': [], 'score': []}
  for t in txt_split:
    res = sentiment(t)
    data["text"].append(t)
    data["label"].append(res[0]['label'])
    data["score"].append(res[0]['score'])

  df = pd.DataFrame(data)
  res = df.groupby('label')['text'].count() / df['label'].count()
  return (res.to_dict(), df)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [ ]:
iface = gr.Interface(fn=get_sentiment,
                    inputs="text",
                    allow_flagging="never",
                    outputs = ["label", "dataframe"],
                    title="Sentiment Analysis",
                    description='Give the sentiment analysis')

In [ ]:
iface.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://44939.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/gradio/routes.py", line 256, in run_predict
    fn_index, raw_input, username, session_state
  File "/usr/local/lib/python3.7/dist-packages/gradio/blocks.py", line 544, in process_api
    predictions, duration = await self.call_function(fn_index, processed_input)
  File "/usr/local/lib/python3.7/dist-packages/gradio/blocks.py", line 459, in call_function
    block_fn.fn, *processed_input, limiter=self.limiter
  File "/usr/local/lib/python3.7/dist-packages/anyio/to_thread.py", line 32, in run_sync
    func, *args, cancellable=cancellable, limiter=limiter
  File "/usr/local/lib/python3.7/dist-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.7/dist-packages/anyio/_backends/_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "/usr/local/lib/python3.7/dist-packages/gradio/interface.py", line 510, in <lam

That's all! Go ahead and open that share link in a new tab. Check out our [getting started](https://gradio.app/getting_started.html) page for more complicated demos.